# Assessing the performance of the model

## Importing

In [63]:
import xarray as xr
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import root_mean_squared_error as rmse


## Datasets Preparation

In [64]:
def datasets_preparation(dataset, dataset2, name):
    
    x = np.tile(dataset.x, len(dataset.time_counter)*len(dataset.y))
    y = np.tile(np.repeat(dataset.y, len(dataset.x)), len(dataset.time_counter))
   
    inputs = np.stack([
        np.ravel(dataset2['Summation_of_solar_radiation']),
        np.ravel(dataset2['Mean_wind_speed']),
        np.ravel(dataset2['Mean_air_temperature']),
        y,
        x,
        np.repeat(dataset.time_counter.dt.dayofyear, len(dataset.x)*len(dataset.y)),
        ])

    targets = np.ravel(dataset[name])
    
    indx = np.where(np.isfinite(targets) & (x>10) & ((x>100) | (y<880)))
    inputs = inputs[:,indx[0]]
    targets = targets[indx[0]]

    inputs = inputs.transpose()

    return(inputs, targets, indx)


## Regressor (Training with all)

In [65]:
def regressor (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4)
    
    # predictions = regr.predict(inputs)
    
    # table[0,0] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    # table[1,0] = rmse(predictions,targets)
    # m,_ = np.polyfit(targets, predictions, deg=1)
    # table[2,0] = np.round(m,3)

    return(regr)


## Regressor (Training with 75%, testing with 25%)

In [66]:
def regressor2 (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    
    X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.25)
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4).fit(X_train,y_train)

    predictions = regr.predict(X_train)

    table[0,2] = np.round(np.corrcoef(y_train,predictions)[0][1],3)
    table[1,2] = rmse(y_train,predictions)
    m,_ = np.polyfit(y_train,predictions, deg=1)
    table[2,2] = np.round(m,3)
    
    predictions = regr.predict(X_test)

    table[0,3] = np.round(np.corrcoef(y_test,predictions)[0][1],3)
    table[1,3] = rmse(y_test,predictions)
    m,_ = np.polyfit(y_test,predictions, deg=1)
    table[2,3] = np.round(m,3)

    return(regr)


## Evaluation

In [67]:
def evaluation (ds,ds2,regr,name, table, i):

    dataset = ds.sel(time_counter = slice('2021', '2024'))
    dataset2 = ds2.sel(time_counter = slice('2021', '2024'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2,name)

    predictions = regr.predict(inputs)

    table[0,i] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,i] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,i] = np.round(m,3)


## Printing

In [68]:
def printing(table,criteria,categories,metric):

    temp = pd.DataFrame(table.transpose(),columns=criteria,index=categories)
    print(metric)
    display(temp)
    print ('\n')

## Training

In [69]:
def training(name,table):

    ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
    ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

    ds = ds.isel(time_counter = (np.arange(0, len(ds.time_counter),2)), 
        y=(np.arange(ds.y[0], ds.y[-1], 5)), 
        x=(np.arange(ds.x[0], ds.x[-1], 5)))

    ds2 = ds2.isel(time_counter = (np.arange(0, len(ds2.time_counter),2)), 
        y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
        x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

    dataset = ds.sel(time_counter = slice('2007', '2020'))
    dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2, name)

    regr = regressor(inputs, targets, table)
    # evaluation(ds,ds2,regr,name,table,1)

    # regr2 = regressor2(inputs, targets, table)
    # evaluation(ds,ds2,regr2,name,table,4)


In [82]:
from sklearn.model_selection import cross_validate

ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

ds = ds.isel(time_counter = (np.arange(0, len(ds.time_counter),2)), 
    y=(np.arange(ds.y[0], ds.y[-1], 5)), 
    x=(np.arange(ds.x[0], ds.x[-1], 5)))

ds2 = ds2.isel(time_counter = (np.arange(0, len(ds2.time_counter),2)), 
    y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
    x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

dataset = ds.sel(time_counter = slice('2007', '2020'))
dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

inputs, targets, _ = datasets_preparation(dataset, dataset2, 'Diatom_Production_Rate')

regr = regressor(inputs, targets, 'Diatom_Production_Rate')
# evaluation(ds,ds2,regr,name,table,1)

# regr2 = regressor2(inputs, targets, table)
# evaluation(ds,ds2,regr2,name,table,4)

test = cross_val_predict(regr,inputs,targets,cv=4)
scores = cross_validate(regr, inputs, targets, cv=4, scoring=('r2', 'neg_root_mean_squared_error'), return_train_score=True)

In [83]:
scores

{'fit_time': array([28.00581884, 27.50227785, 31.47984838, 15.18430567]),
 'score_time': array([5.45695806, 5.44290614, 3.96375799, 5.62351942]),
 'test_r2': array([0.76540508, 0.75882021, 0.70923635, 0.64670963]),
 'train_r2': array([0.89467319, 0.89150881, 0.89964414, 0.90768806]),
 'test_neg_root_mean_squared_error': array([-7.20394450e-07, -7.74626654e-07, -8.30629665e-07, -8.33086223e-07]),
 'train_neg_root_mean_squared_error': array([-4.92702930e-07, -4.89198446e-07, -4.75879529e-07, -4.67593536e-07])}

In [84]:
(np.corrcoef(targets,test)[0,1])


0.8539627125646575

array([ 0.23773701,  0.30103952,  0.18705661,  0.08482825, -0.10614982])

In [73]:
np.mean(np.abs(scores['test_r2']))

0.726074190927017

## Evaluation

In [ ]:
criteria = ['r','rms','slope']
categories = ['training with 100%', 'testing', 'training with 75%', 'testing with 25%', 'testing']

diat = np.zeros((len(criteria),len(categories)))
flag = np.zeros((len(criteria),len(categories)))
diat_pr = np.zeros((len(criteria),len(categories)))
flag_pr = np.zeros((len(criteria),len(categories)))

test = training('Diatom',diat)
# training('Flagellate',flag)
# training('Diatom_Production_Rate',diat_pr)
# training('Flagellate_Production_Rate',flag_pr)


## Printing (Results)

In [ ]:
printing(diat,criteria, categories,'Diatom')
printing(flag,criteria, categories,'Flagellate')
printing(diat_pr,criteria, categories,'Diatom production rate')
printing(flag_pr,criteria, categories, 'Flagellate production rate')
